In [1]:
# compare cosine similarity of two models

import torch
import tensorflow as tf
import numpy as np
import torch.nn.functional as F
import torch.nn as nn
from tensorflow.keras.models import load_model

2026-01-20 10:11:12.064883: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-20 10:11:12.167207: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-20 10:11:12.598039: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.4/lib64
2026-01-20 10:11:12.598081: W tensorflow/compiler/xla/stream_executor/p

In [2]:
def get_gradients(states):
    """Calculates gradients for both models on a batch of states."""
    
    # --- PyTorch Gradient ---
    pt_states = torch.tensor(states, dtype=torch.float32, requires_grad=True)
    pt_outputs = pytorch_surrogate(pt_states)
    
    # We take the gradient of the max output (the chosen action's logit)
    # This mimics what JSMA looks at
    pt_target = pt_outputs.max(dim=1)[0].sum()
    pt_target.backward()
    pt_grads = pt_states.grad.detach().numpy()

    # --- Keras Gradient ---
    tf_states = tf.convert_to_tensor(states, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(tf_states)
        tf_outputs = keras_policy(tf_states)
        tf_target = tf.reduce_max(tf_outputs, axis=1)
        
    tf_grads = tape.gradient(tf_target, tf_states).numpy()
    
    return pt_grads, tf_grads


def cosine_sim(a, b):
    # Flatten to vectors for similarity calculation
    a_flat = a.reshape(a.shape[0], -1)
    b_flat = b.reshape(b.shape[0], -1)
    
    dot_product = np.sum(a_flat * b_flat, axis=1)
    norms = np.linalg.norm(a_flat, axis=1) * np.linalg.norm(b_flat, axis=1)
    return dot_product / (norms + 1e-8)

In [15]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(26, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = F.softmax(x, dim=1)
        return x

device = torch.device( "cpu") #"cuda" if torch.cuda.is_available() else
pytorch_surrogate = BinaryClassifier().to(device)

# Load the saved weights
pytorch_surrogate.load_state_dict(torch.load("binary_classifier.pth", map_location=torch.device('cpu')))
# Set to evaluation mode
pytorch_surrogate.eval()

BinaryClassifier(
  (fc1): Linear(in_features=26, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)

In [16]:
def softmax_temp(x):
    # ref: https://stackoverflow.com/questions/63471781/making-custom-activation-function-in-tensorflow-2-0
    # ref: http://www.kasimte.com/2020/02/14/how-does-temperature-affect-softmax-in-machine-learning.html
    # ref: https://jamesmccaffrey.wordpress.com/2016/03/04/the-max-trick-when-computing-softmax/
    e_x = tf.exp(tf.divide(x - tf.reduce_max(x, axis=-1, keepdims=True), 1.0))
    output = tf.divide(e_x, tf.reduce_sum(e_x, axis=-1, keepdims=True))
    return output

In [17]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

input_d = 26
output_d = 2
hidden_d = [26*3,26*3]
hidden_act = 'elu'

model_in = Input(shape=(input_d,), name='state_in')
#advantage = Input(shape=(1,), name='advantage')
q_values = Input(shape=(output_d,), name='q_values')
sampled_actions = Input(shape=(output_d,), name='sampled_actions')

layers = {}
# we receive WARNING:tensorflow:Output dense_2 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to dense_2.
# but it seems not influence the result
for i in range(len(hidden_d)):
    if i == 0:
        layers[i] = Dense(hidden_d[i], activation=hidden_act, kernel_initializer='he_uniform')(model_in)
    else:
        layers[i] = Dense(hidden_d[i], activation=hidden_act, kernel_initializer='he_uniform')(layers[i-1])

dropout_layer = Dropout(0.5,input_shape=(hidden_d[i],))(layers[len(hidden_d)-1])
model_out = Dense(output_d, activation=softmax_temp, kernel_initializer='he_uniform')(dropout_layer)

final_model = Model(model_in, model_out)
final_model.compile(optimizer='adam', loss='categorical_crossentropy')

In [19]:
final_model.load_weights('79_10000.h5') 

# Use 'final_model' directly from here on
keras_policy = final_model



# 2. Run Verification on 100 Sample States
sample_states = np.random.rand(100, 26).astype(np.float32) # Replace with real states
pt_grads, tf_grads = get_gradients(sample_states)

# 3. Calculate Cosine Similarity


similarities = cosine_sim(pt_grads, tf_grads)

print(f"Mean Gradient Cosine Similarity: {np.mean(similarities):.4f}")
print(f"Max/Min Similarity: {np.max(similarities):.4f} / {np.min(similarities):.4f}")

Mean Gradient Cosine Similarity: -0.0070
Max/Min Similarity: 0.9034 / -0.9010


In [20]:
def sync_weights(keras_model, pytorch_model):
    """
    Copies weights from a Keras model to a PyTorch model, handling
    the necessary matrix transposition for Linear/Dense layers.
    """
    print("Syncing weights...")
    
    # Get all Keras layers that have weights (Dense layers)
    keras_dense_layers = [l for l in keras_model.layers if 'dense' in l.name]
    
    # Get all PyTorch Linear layers
    # Note: This assumes the order of layers matches exactly!
    pytorch_linear_layers = [m for m in pytorch_model.modules() if isinstance(m, torch.nn.Linear)]

    if len(keras_dense_layers) != len(pytorch_linear_layers):
        raise ValueError(f"Layer count mismatch! Keras: {len(keras_dense_layers)}, PyTorch: {len(pytorch_linear_layers)}")

    with torch.no_grad():
        for i, (k_layer, pt_layer) in enumerate(zip(keras_dense_layers, pytorch_linear_layers)):
            # Keras returns [weights, biases]
            k_weights, k_bias = k_layer.get_weights()
            
            # --- CRITICAL STEP: TRANSPOSE WEIGHTS ---
            # Keras: (In, Out) -> PyTorch: (Out, In)
            pt_layer.weight.data = torch.tensor(k_weights.T).float()
            
            # Biases are usually 1D and don't need transposing
            pt_layer.bias.data = torch.tensor(k_bias).float()
            
            print(f"Layer {i}: Synced Keras shape {k_weights.shape} to PyTorch shape {pt_layer.weight.shape}")

    print("Sync complete.")

# 1. Run the sync
sync_weights(keras_policy, pytorch_surrogate)

# 2. Re-run your verification
pt_grads, tf_grads = get_gradients(sample_states)
similarities = cosine_sim(pt_grads, tf_grads)
print(f"New Mean Similarity: {np.mean(similarities):.4f}")

Syncing weights...
Layer 0: Synced Keras shape (26, 78) to PyTorch shape torch.Size([78, 26])
Layer 1: Synced Keras shape (78, 78) to PyTorch shape torch.Size([78, 78])
Layer 2: Synced Keras shape (78, 2) to PyTorch shape torch.Size([2, 78])
Sync complete.


RuntimeError: Function TBackward0 returned an invalid gradient at index 0 - got [2, 78] but expected shape compatible with [2, 32]